In [21]:
import anuga.parallel.mpiextras

ImportError: No module named mpiextras

In [10]:
import sys
sys.path.append('/home/ZhiLi/CRESTHH/anuga_core/anuga/parallel')
import anuga
from osgeo import gdal
from glob import glob
from affine import Affine
import geopandas as gpd
import pypar

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')
#  Mesh partition routines


# from anuga.parallel.parallel_meshes import parallel_rectangle
from anuga import distribute, myid, numprocs, finalize, barrier

In [12]:
bounding_polygon= gpd.read_file('area/domain.shp')

# Elevation Data
topography_file = 'DEM.tif'
# topography_file= '/home/ZhiLi/CRESTHH/data/precip/imerg20170825S150000.tif'

# Resolution for most of the mesh
base_resolution = 50.0  # m^2




In [13]:
if myid==0:
    domain = anuga.create_domain_from_regions(
            list(bounding_polygon.exterior[0].coords),
            boundary_tags={'bottom': [0],},
            maximum_triangle_area=base_resolution,
            )

    domain.set_name('excessive_rain_para') # Name of sww file

    domain.set_quantity('elevation', filename=topography_file, location='centroids') # Use function for elevation
    domain.set_quantity('friction', 0.01, location='centroids')                        # Constant friction 
    domain.set_quantity('stage', expression='elevation', location='centroids')         # Dry Bed 

    Br = anuga.Reflective_boundary(domain)
    Bt = anuga.Transmissive_boundary(domain)

    domain.set_boundary({'bottom':   Bt,
                         'exterior': Br})
else:
    domain=None

In [14]:
domain= distribute(domain)

In [17]:
external_dir= '/home/ZhiLi/CRESTHH/data/precip'
Rain = anuga.Quantity(domain, name='excessive rain')
rain_op = anuga.Rate_operator(domain, rate = Rain, factor=10)
for i,t in enumerate(domain.evolve(yieldstep=1, duration=20)):
    current_rain_file = sorted(glob(external_dir+'/*.tif'))[i]
    print domain.timestepping_statistics()
    Rain.set_values_from_utm_grid_file(current_rain_file, location='centroids')
domain.sww_merge(verbose=True)

Time = 2.0000, delta t = 705.61713004, steps=1 (33s)
Time = 3.0000, delta t = 705.61713004, steps=1 (0s)


KeyboardInterrupt: 